# Day 1

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('input.txt', header=None)
x = df.iloc[:,0]

## Part one

In [3]:
((x - x.shift(1).bfill()).values.flatten() > 0).sum() 

1832

## Part two

In [4]:
wx = x.rolling(window=3).sum().dropna()
((wx - wx.shift(1).bfill()).values.flatten() > 0).sum() 

1858

# Day 2

In [13]:
df = pd.read_csv('input2.txt', header=None, delimiter=' ')
df.columns = ['direction', 'val']
df.direction.unique()

array(['forward', 'down', 'up'], dtype=object)

## Part one

In [17]:
forward = df[df.direction == 'forward'].val.sum()
down = df[df.direction == 'down'].val.sum()
up = df[df.direction == 'up'].val.sum()
forward * (down-up)

1936494

## Part two

In [19]:
hori, aim, depth = 0, 0, 0
for cmd, val in df.values:
    if cmd == 'down':
        aim += val
    if cmd == 'up':
        aim -= val
    if cmd == 'forward':
        hori += val
        depth += aim*val
hori*depth

1997106066

# Day 3

In [ ]:
df = pd.read_csv('input3.txt', header=None)
df.columns = ['raw']
n_bits = df.raw
df